In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from time import sleep
from urllib.parse import quote
from queue import Queue
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
import requests

In [6]:
# Função para acessar o portal com os parametros desejados
def consultar(navegador):
    url='https://transparencia.mprj.mp.br/web/novo-portal-transparencia/processos-distribuidos-novo'
    navegador.get(url)

In [7]:
# Iniciando drive do Chorme
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver)
sleep(5)

In [28]:
# Identificadores do HTML

comarca_id = 'comarcaId'
orgao_id = 'orgaoId'
tipo_documento_id = 'documentoId'
membro_id = 'membroId'

btn_buscar_id = 'export'

linha_tabela_class = 'tr-modal' #'tabela-cargos-membros'
pagination_css = 'li.page-item.active'

def get_element(id):
    element = driver.find_element(By.ID, id)
    return element


In [29]:
col_comarca = []
col_orgao = []
col_tipo_doc = []
col_membro = []
col_numero = []
col_distribuido_em = []
col_submetido_em = []
col_tipo = []
col_unidade = []    

comarcas = get_element(comarca_id)

for comarca in comarcas.find_elements(By.TAG_NAME, "option"):
    value = comarca.get_attribute('value')

    if value:
        Select(comarcas).select_by_value(value)
    else:
        continue

    name_comarca = comarca.text
    sleep(1)

    orgaos = get_element(orgao_id)
    for orgao in orgaos.find_elements(By.TAG_NAME, "option"):
        value = orgao.get_attribute('value')
        if value:
            Select(orgaos).select_by_value(value)  
        else:
            continue   

        name_orgao = orgao.text
        sleep(1)

        tipos = get_element(tipo_documento_id)
        for tipo in tipos.find_elements(By.TAG_NAME, "option"):
            value = tipo.get_attribute('value')
            if value:
                Select(tipos).select_by_value(value)  
            else:
                continue    

            name_tipo_doc = tipo.text
            sleep(1)

            membros = get_element(membro_id)
            for membro in membros.find_elements(By.TAG_NAME, "option"):
                value = membro.get_attribute('value')
                if value:
                    Select(membros).select_by_value(value)     
                    # print('membro encontrado')
                else:
                    # print('membro nao encontrado')
                    continue

                name_membro = membro.text
                                    
                # Busca
                buscar = get_element(btn_buscar_id).click() 
   
                sleep(1)          

                pages_visited = []

                actual_page = driver.find_element(By.CSS_SELECTOR, pagination_css)   
                # pages_visited.append(actual_page.text)

                next_pages = actual_page.find_elements(By.XPATH, './/following-sibling::li')


                for i in range(0, len(next_pages)):
                    # print(next_pages[i].get_attribute('innerHTML'))

                    if next_pages[i].text in pages_visited:
                        continue
                    else:
                        linhas_tabela = driver.find_elements(By.CLASS_NAME, linha_tabela_class)  

                        for linha in linhas_tabela:     
                            # print(linha.get_attribute('innerHTML'))
                            
                            try:
                                colunas = linha.find_elements(By.TAG_NAME, 'td')
                            except Exception:
                                break

                            colunas = list(colunas)

                            # print(list(colunas)[0].text)
                            # print(colunas[1].get_attribute('text'))

                            # print(len(linhas_tabela))

                            col_numero.append(colunas[0].text)
                            col_distribuido_em.append(colunas[1].text)
                            col_submetido_em.append(colunas[2].text)
                            col_tipo.append(colunas[3].text)
                            col_unidade.append(colunas[4].text)   
                            col_comarca.append(name_comarca)
                            col_orgao.append(name_orgao)
                            col_tipo_doc.append(name_tipo_doc)
                            col_membro.append(name_membro)

                    pages_visited.append(next_pages[i].text)
                    next_pages[i].click()                  

                driver.find_element(By.CSS_SELECTOR, 'button.close.btn.btn-unstyled').click() #fechar modal

                sleep(2)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D93BEEA2+31554]
	(No symbol) [0x00007FF7D9337ED9]
	(No symbol) [0x00007FF7D91F872A]
	(No symbol) [0x00007FF7D920B4E7]
	(No symbol) [0x00007FF7D920B84A]
	(No symbol) [0x00007FF7D91FF5BB]
	(No symbol) [0x00007FF7D91FD326]
	(No symbol) [0x00007FF7D9200EB1]
	(No symbol) [0x00007FF7D9200F50]
	(No symbol) [0x00007FF7D92421DB]
	(No symbol) [0x00007FF7D926D02A]
	(No symbol) [0x00007FF7D923BA76]
	(No symbol) [0x00007FF7D926D240]
	(No symbol) [0x00007FF7D928C977]
	(No symbol) [0x00007FF7D926CDD3]
	(No symbol) [0x00007FF7D923A33B]
	(No symbol) [0x00007FF7D923AED1]
	GetHandleVerifier [0x00007FF7D96C8B1D+3217341]
	GetHandleVerifier [0x00007FF7D9715AE3+3532675]
	GetHandleVerifier

In [39]:
actual_page.get_attribute('innerHTML')

'<a class="page-link" aria-label="Go to page, 1" tabindex="0">1</a>'

In [50]:
actual_page.find_elements(By.XPATH, './/following-sibling::li')[-1].get_attribute('innerHTML')

'<a class="page-link" aria-label="Go to the next page, 5" role="button" tabindex="0"><svg class="lexicon-icon lexicon-icon-angle-right" role="presentation" viewBox="0 0 512 512"><g>\n\t<path class="lexicon-icon-outline" d="M396.394 255.607c-0.22-6.936-2.973-13.81-8.272-19.111l-227.221-227.221c-11.026-11.059-28.94-11.059-39.999 0-11.058 11.026-11.058 28.941 0 39.999l206.333 206.333c0 0-206.333 206.333-206.333 206.333-11.058 11.058-11.058 28.973 0 39.999 11.059 11.059 28.972 11.059 39.999 0l227.221-227.221c5.3-5.3 8.053-12.175 8.272-19.111z"></path>\n</g></svg></a>'

In [41]:
next_pages

[<selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8bf4c55be966d2b54c8", element="f.7D463F2E34798FDE215243054345D32F.d.FB75E382808B54E8DED3F87672C2D0CA.e.2265")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8bf4c55be966d2b54c8", element="f.7D463F2E34798FDE215243054345D32F.d.FB75E382808B54E8DED3F87672C2D0CA.e.2266")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8bf4c55be966d2b54c8", element="f.7D463F2E34798FDE215243054345D32F.d.FB75E382808B54E8DED3F87672C2D0CA.e.2267")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8bf4c55be966d2b54c8", element="f.7D463F2E34798FDE215243054345D32F.d.FB75E382808B54E8DED3F87672C2D0CA.e.2268")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8bf4c55be966d2b54c8", element="f.7D463F2E34798FDE215243054345D32F.d.FB75E382808B54E8DED3F87672C2D0CA.e.2269")>,
 <selenium.webdriver.remote.webelement.WebElement (session="c34cc0a8727af8b

In [42]:
next_pages[i].get_attribute('innerHTML')

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D93BEEA2+31554]
	(No symbol) [0x00007FF7D9337ED9]
	(No symbol) [0x00007FF7D91F872A]
	(No symbol) [0x00007FF7D920B4E7]
	(No symbol) [0x00007FF7D920B84A]
	(No symbol) [0x00007FF7D91FF5BB]
	(No symbol) [0x00007FF7D920B888]
	(No symbol) [0x00007FF7D91FF5BB]
	(No symbol) [0x00007FF7D91FD326]
	(No symbol) [0x00007FF7D92011D8]
	(No symbol) [0x00007FF7D928D69B]
	(No symbol) [0x00007FF7D926D02A]
	(No symbol) [0x00007FF7D928C977]
	(No symbol) [0x00007FF7D926CDD3]
	(No symbol) [0x00007FF7D923A33B]
	(No symbol) [0x00007FF7D923AED1]
	GetHandleVerifier [0x00007FF7D96C8B1D+3217341]
	GetHandleVerifier [0x00007FF7D9715AE3+3532675]
	GetHandleVerifier [0x00007FF7D970B0E0+3489152]
	Get

In [24]:
scraping_values = {'Comarca': col_comarca, 'Orgão de execução': col_orgao, 'Tipo de documento': col_tipo, 'Membro': col_membro, 'Número': col_numero, 'Distribuído em': col_distribuido_em, 'Submetido em': col_submetido_em, 'Tipo': col_tipo, 'Unidade': col_unidade}
# scraping_values

In [27]:
df = pd.DataFrame.from_dict(scraping_values) # columns=['Número', 'Distribuído em', 'Submetido em', 'Tipo', 'Unidade'])
df

,Comarca,Orgão de execução,Tipo de documento,Membro,Número,distribuido_em,submetido_em,tipo,unidade
0,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08050941320228190003,12/07/2024,12/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
1,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08097853620238190003,12/07/2024,12/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
2,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,00096019420158190003,12/07/2024,12/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
3,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08071542220238190003,12/07/2024,12/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
4,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,00071647020218190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
5,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08035095220248190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
6,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08044388520248190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
7,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08089054420238190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
8,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08011729020248190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...
9,ANGRA DOS REIS,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...,Processo judicial de 1ª Instância,PLINIO VINICIUS D´AVILA ARAUJO,08087902320238190003,11/07/2024,11/07/2024,Processo judicial de 1ª Instância,PROMOTORIA DE JUSTIÇA CÍVEL E DE FAMÍLIA DE AN...


## Esboço do marcos

In [ ]:
# Função para pegar os links do botão detalhar
def get_href(elements):
    hrefs = []
    for elemento in elements:
        href = elemento.get_attribute('href')
        hrefs.append(href)

    return hrefs

# Função para acessar o portal com os parametros desejados
def consultar(navegador, beneficio, start_date, end_date, municipio = 'Tanguá'):
    municipio = quote(municipio)
    navegador.get(f'https://portaldatransparencia.gov.br/beneficios/{beneficio}?de={start_date}&ate={end_date}&nomeMunicipio={municipio}&ordenarPor=nis&direcao=asc')

# Função para verificar se o número da página é maior que o valor anteriormente armazenado
def page_number_greater(driver, previous_page, id = 'lista_info'):
    page_text = driver.find_element(By.ID, id).text
    current_page = int(page_text.split()[1])
    return current_page > previous_page

In [ ]:
beneficio = 'bolsa-familia'
start_date = '01/01/2020'
end_date = '31/12/2020'
municipio = 'Tanguá'

# Iniciando drive do Chorme
driver = webdriver.Chrome()

# inicia a pagiana e espera 5 segundos para os dados serem carregados
consultar(driver, beneficio, start_date, end_date, municipio)
sleep(5)

# Clica no botão de ver paginação completa
driver.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
sleep(5)

# Seleciona a opção de 50 individuos por páginas
elemento_dropdown  = driver.find_element(By.NAME, 'lista_length')
Select(elemento_dropdown).select_by_value('50')
sleep(5)

# Verifica quantas paginas no total
quantPaginas = int(driver.find_element(By.ID, 'lista_info').text.split()[-1])

# Crie uma lista para armazenar os valores de atributo href
hrefs = []
previous_page = 1
for _ in range(quantPaginas):
    # Busca os lnkes do detalhar
    elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')

    # Tenta obeter o link, caso a requisição demore para responder, espere mais 5 segundos
    try:
        link = get_href(elements)
    except:
        sleep(5)
        elements = driver.find_elements(By.CLASS_NAME, 'linkRendered')
        link = get_href(elements)


    # Adiciona na lista de links
    hrefs.extend(link)
    if previous_page != quantPaginas:
        # Clica no botão proximo  
        driver.find_element(By.ID, 'lista_next').click()

        # Aguardar a mudança de página
        WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page))
        previous_page += 1

driver.quit()

## Transformando links em csv

In [ ]:
df = pd.DataFrame(hrefs, columns=['Detalhes_link'])

In [ ]:
ano = start_date.split('/')[-1]

df.to_csv(f'{start_date}.csv')

In [ ]:
df = pd.read_csv('2017.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv('bolsa-familia-2017.csv')

In [ ]:
# Lista para armazenar os dados dos arquivos CSV
dados_csv = []

# Percorra todos os arquivos na pasta
for nome_arquivo in os.listdir('Dados/'):
    if nome_arquivo.endswith('.csv'):
        # Construa o caminho completo para cada arquivo CSV
        caminho_arquivo = os.path.join('Dados/', nome_arquivo)
        
        # Leia o arquivo CSV e adicione os dados à lista
        df = pd.read_csv(caminho_arquivo)
        dados_csv.append(df)

# Concatene todos os DataFrames em um único DataFrame
dados_completos = pd.concat(dados_csv)

In [ ]:
dados_completos.drop('Unnamed: 0', axis=1, inplace=True)
dados_completos.drop_duplicates(inplace=True)
dados_completos.reset_index(drop=True, inplace=True)
dados_completos.to_csv('Dados/bolsa-familia.csv', index=False)

# Teste de extrair quantidade da tabela

In [ ]:
driver = webdriver.Chrome()
driver.get('https://portaldatransparencia.gov.br/beneficios/bolsa-familia/301738618?ordenarPor=mesReferencia&direcao=desc')
sleep(3)
user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
name = user_data.get_attribute("innerHTML").split()[-1]


In [ ]:
df = pd.read_csv('Dados/bolsa-familia.csv')
link_list = df.values.tolist()

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
error_link = []
dados = []

# Percorre cada link
for link in link_list:
    driver.get(link[0])
    sleep(3)

    # Localize o NIS
    try:
        user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]
    except:
        error_link.append(link[0])  # Pegua o link que deu erro
        driver.quit()               # Finaliza a instancia do chorme
        sleep(60)                   # Aguarda caso seja bloqueio do sistema
        driver = webdriver.Chrome() # Cria uma nova instancia
        driver.maximize_window()    # Maximiza a tela
        driver.get(link[0])         # Acessa o link
        sleep(3)                    # Aguarda pagina carregar
        user_data = driver.find_element(By.CLASS_NAME, 'dados-tabelados').find_element(By.CLASS_NAME, 'row').find_elements(By.TAG_NAME, 'div')[-1]

    nis = user_data.get_attribute("innerHTML").split()[-1]

    # Localize a tabela geral
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_wrapper')
    tabela.find_element(By.CLASS_NAME, 'botao__gera_paginacao_completa').click()
    sleep(5)

    # Seleciona a opção de 50 individuos por páginas
    elemento_dropdown  = driver.find_element(By.NAME, 'tabelaDetalheValoresRecebidos_length')
    Select(elemento_dropdown).select_by_value('50')
    sleep(5)

    # Verifica quantas paginas no total
    quantPaginas = int(driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').text.split()[-1])
    
    # Pega a tabela principal
    tabela = driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos')
    
    previous_page = 1
    for _ in range(quantPaginas):

        # Gambiarra para tirar o mouse de tootip em cima dos dados
        driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_info').click()
        sleep(0.1)

        # Obtenha todas as linhas da tabela
        linhas = tabela.find_elements(By.TAG_NAME, 'tr')[1:]

        # Itere sobre as linhas
        for linha in linhas:
            # Obtenha todas as células da linha
            celulas = linha.find_elements(By.TAG_NAME, 'td')
            
            # Extraia os dados das células
            dados_linha = [celula.text for celula in celulas]

            # Adicionando o nis
            dados_linha.append(nis)

            # Armazene os dados
            dados.append(dados_linha)

        if previous_page != quantPaginas:
            # Clica no botão proximo  
            driver.find_element(By.ID, 'tabelaDetalheValoresRecebidos_next').click()

            # Aguardar a mudança de página
            WebDriverWait(driver, 10).until(lambda driver: page_number_greater(driver, previous_page, 'tabelaDetalheValoresRecebidos_info'))
            previous_page += 1
        
# Feche o navegador
driver.quit()

peguei


KeyboardInterrupt: 

In [ ]:
# Começar partir daqui
linha = df[df['Detalhes_link'] == 'https://portaldatransparencia.gov.br/beneficios/bolsa-familia/299403958'].index[0]

In [ ]:
linha

697

In [ ]:
link_list[linha:]

In [ ]:
header = ['Mes Folha', 'Mes Referência', 'UF', 'Municipio', 'Quantidade Dependentes', 'Valor', 'NIS']
pd.DataFrame(dados, columns=header).to_csv('Dados/detalhes-bolsa-familia.csv')

# Usando API

In [ ]:
def obter_dados_auxilio_emergencial(codigo_ibge, mes_ano, beneficio):
    url = f"https://api.portaldatransparencia.gov.br/api-de-dados/{beneficio}-disponivel-beneficiario-por-municipio"
    headers = {"chave-api-dados": "07e464f446e88bb0c105f9cea69b7f45"}

    page = 1
    dados = []

    while True:
        params = {
            "codigoIbge": codigo_ibge,
            "mesAno": mes_ano,
            "pagina": str(page)
        }

        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        for item in data:
            dados.append([
                item['dataMesReferencia'],
                item['municipio']['nomeIBGE'],
                item['titularBolsaFamilia']['cpfFormatado'],
                item['titularBolsaFamilia']['nis'],
                item['titularBolsaFamilia']['nome'],
                item['quantidadeDependentes'],
                item['valor']
            ])

        page += 1
    return dados

In [ ]:
header = ['Mes Referência', 'Município', 'CPF Beneficiário', 'NIS Beneficiário', 'Beneficiário', 'Quantidade Dependentes', 'Valor']

# Utilização da função para obter os dados para todos os meses do ano
try:
    codigo_ibge = "3305752"
    beneficio = 'bolsa-familia'
    anos = [str(ano) for ano in range(2016,2024)] # Adicione todos os anos que deseja incluir
    meses = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]  # Todos os meses do ano

    for ano in anos:
        for mes in meses:
            mes_ano = ano + mes
            dados = obter_dados_auxilio_emergencial(codigo_ibge, mes_ano, beneficio)
            
            # Se os dados não estiverem vazios
            if dados:
                # Cria a pasta para o ano, caso ela não exista
                pasta_ano = f"Dados/{beneficio}/{ano}"
                if not os.path.exists(pasta_ano):
                    os.makedirs(pasta_ano)

                # Salva os dados em um arquivo CSV
                nome_arquivo = f"{mes_ano}_{beneficio}.csv"
                caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)
                df = pd.DataFrame(dados, columns=header)
                df.to_csv(caminho_arquivo, index=False)
                print(f"Dados para o mês {mes} do ano {ano} foram salvos.")
            else:
                print(f"Dados para o mês {mes} do ano {ano} estava vazio.")
            

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err}")
except Exception as err:
    print(f"Erro: {err}")

In [ ]:
import os
import pandas as pd

In [ ]:
# Cria uma lista para armazenar os DataFrames de todos os anos
dfs_anuais = []
diretorio_principal = f"Dados/Por_Beneficiario/Auxilio_Emergencial/"
beneficio = 'AuxilioEmergencial'


# Loop para processar cada pasta de ano
for ano in range(2013, 2023):
    pasta_ano = os.path.join(diretorio_principal, str(ano))
    
    # Verifica se a pasta do ano existe
    if not os.path.exists(pasta_ano):
        print(f"A pasta {ano} não existe. Pulando para o próximo ano.")
        continue

    # Cria uma lista para armazenar os DataFrames de cada mês
    dfs_mensais = []

    # Loop para processar os 12 arquivos de cada mês
    for mes in range(1, 13):
        nome_arquivo = f"{ano}{mes:02d}_{beneficio}.csv"
        caminho_arquivo = os.path.join(pasta_ano, nome_arquivo)

        # Verifica se o arquivo do mês existe
        if not os.path.exists(caminho_arquivo):
            print(f"O arquivo {nome_arquivo} não existe. Pulando para o próximo mês.")
            continue

        # Lê o arquivo CSV do mês e adiciona ao DataFrame
        df_mensal = pd.read_csv(caminho_arquivo)
        dfs_mensais.append(df_mensal)

    # Concatena os DataFrames mensais em um único DataFrame anual
    df_anual = pd.concat(dfs_mensais)

    # Adiciona o DataFrame anual à lista de DataFrames de todos os anos
    dfs_anuais.append(df_anual)

# Concatena os DataFrames anuais em um único DataFrame para todos os anos
df_final = pd.concat(dfs_anuais)

# Salva o DataFrame final como um arquivo CSV no diretório de saída
arquivo_saida = os.path.join(diretorio_principal, f"Todos_Anos_{beneficio}.csv")
df_final.to_csv(arquivo_saida, index=False)
print(f"Arquivo compilado todos_anos_{beneficio}_compilado.csv salvo com sucesso.")

print("Processo concluído.")


O arquivo 202001_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202002_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202003_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202202_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202203_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202205_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202206_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202209_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202210_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202211_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
O arquivo 202212_AuxilioEmergencial.csv não existe. Pulando para o próximo mês.
Arquivo compilado todos_anos_AuxilioEmergencial_compilado.csv salvo com sucesso.
Processo concluído.


In [ ]:
import csv
import os
import pandas as pd

In [ ]:
# Arquivos compilados para diferentes benefícios
arquivos_beneficios = [
    "Todos_anos_BolsaFamilia.csv",
    "Todos_Anos_AuxilioBrasil.csv",
    "Todos_Anos_bpc.csv",
    "Todos_Anos_AuxilioEmergencial.csv"
]

# Lista para armazenar os DataFrames de cada benefício
dfs_beneficios = []



# Loop para ler cada arquivo e criar um DataFrame para cada benefício
for arquivo in arquivos_beneficios:
    beneficio = os.path.splitext(arquivo)[0].split('_')[2]  # Obtém o nome do benefício do nome do arquivo
    df_beneficio = pd.read_csv("Dados/Por_Beneficiario/"+arquivo)
    df_beneficio['Benefício'] = beneficio  # Adiciona a coluna "Benefício"
    dfs_beneficios.append(df_beneficio)

# Juntar os DataFrames de todos os benefícios em um único DataFrame
df_final = pd.concat(dfs_beneficios, join='outer', ignore_index=True)

# Converter a coluna "NIS Beneficiário" para string com aspas duplas
# df_final["NIS Beneficiário"] = df_final["NIS Beneficiário"].apply(lambda x: f'"{x}"')

# Converter a coluna "NIS Beneficiário" para string
df_final["NIS Beneficiário"] = df_final["NIS Beneficiário"].astype(str)


# Salvar o DataFrame final como um arquivo CSV
df_final.to_csv("Todos_Anos_Todos_Beneficios.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)
print("Arquivo compilado todos_anos_todos_beneficios_compilado.csv salvo com sucesso.")


Arquivo compilado todos_anos_todos_beneficios_compilado.csv salvo com sucesso.


In [ ]:
import pandas as pd

In [ ]:
df =pd.read_csv('Todos_Anos_Todos_Beneficios.csv')
df['NIS Beneficiário'] = df["NIS Beneficiário"].astype(str)

C:\Users\MARCOS~1\AppData\Local\Temp/ipykernel_19464/3809605594.py:1: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df =pd.read_csv('Todos_Anos_Todos_Beneficios.csv')


In [ ]:
beneficios_procurados = ['BolsaFamilia', 'AuxilioBrasil', 'AuxilioEmergencial', 'bpc']
df_filtered = df[df['Benefício'].isin(beneficios_procurados)]

nis_with_multiple_benefits = df_filtered.groupby('NIS Beneficiário')['Benefício'].nunique()

nis_with_multiple_benefits = nis_with_multiple_benefits[nis_with_multiple_benefits > 1]

nis_with_multiple_benefits

NIS Beneficiário
10083059072.0    2
10090276873.0    2
10102942029.0    3
10103053295.0    4
10119460812.0    4
                ..
23880183771.0    3
23880617739.0    3
23883036273.0    2
23891574270.0    2
23893787468.0    3
Name: Benefício, Length: 4842, dtype: int64